In [64]:
import numpy as np
import math
import random
import matplotlib.pyplot as plt
import pandas as pd
import copy
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from scipy import stats
from sklearn.metrics import confusion_matrix
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from pandas.testing import assert_frame_equal
import sys

In [65]:
#加载数据 预处理
data = pd.read_csv("../data/gene_version2020/generate/ExperimentalDatasets/MM/MM-BP.csv",index_col=0)
label = data["longevity influence"]
#删除无用列
data = data.drop(["longevity influence"],axis=1)

#初始化基因本体的DAG
DAG = pd.read_csv("../data/gene_version2020/generate/GOPath/MM/MM-BP.csv",index_col=0)

#过滤低纬度特征
for c in data.columns:
    data[c] = data[c].astype('int')
    if(data[c].sum() < 3):
        data.drop(c,axis=1,inplace=True)
DAG = DAG.loc[data.columns,data.columns]

#计算相关性矩阵
SUMetrics = pd.DataFrame(0,columns=data.columns,index=data.columns)


elite = 4  #精英保留数量
k = 6 #tournament选择个数
j = 0 #本轮进化已经挑选个体数
G = 149  #G:终止进化次数
Pc = 0.95  #Pc:交叉发生概率
Pm_standard = 0.06  #变异发生概率
SHEP = 0.19 #SHE算法突变概率
num = 146  #num:种群生物数量
d = data.shape[1]  #特征维度
relativeList = [] #用于递归中存储祖先和后代节点

In [74]:
def RWS(x): #轮盘赌算法，根据概率选出个体
    m = 0
    r = random.random()
    for i in range(num):
        m = m + x[i]
        if(r <= m):
            return i
def draw(x): #绘制结果
    x_axis = np.arange(-1,2,0.000001)
    y_axis = x_axis * np.sin(x_axis*10*math.pi) + 2
    
    h = np.zeros(num)
    y = np.zeros(num)
    for i in range(num):#计算每个个体的适应度
        #根据二进制计算十进制
        h[i] = decoding(x[i]) #解码
        y[i] = fitness(h[i])
    plt.rcParams['figure.figsize'] = (15, 8)
    plt.plot(x_axis,y_axis)
    plt.scatter(h,y,c='red')
    #print(h)
    plt.show()
def decoding(x): #解码
    # 转换算子
    Bi_conver_op=2**np.arange(x.shape[0])
    b=x.dot(Bi_conver_op[::-1].T)
    return b*3/(2**22-1)-1


def Entropy(labels,base=2): #计算信息熵
    # 计算概率分布
    probs = pd.Series(labels).value_counts() / len(labels)
    # 计算底数为base的熵
    en = stats.entropy(probs, base=base)
    return en
def calc_ent(x):

    x_value_list = set([x[i] for i in range(x.shape[0])])
    ent = 0.0
    for x_value in x_value_list:
        p = float(x[x == x_value].shape[0]) / x.shape[0]
        logp = np.log2(p)
        ent -= p * logp

    return ent
def calc_condition_ent(x, y): #计算条件熵
    # calc ent(y|x)
    x_value_list = set([x[i] for i in range(x.shape[0])])
    ent = 0.0
    for x_value in x_value_list:
        sub_y = y[x == x_value]
        temp_ent = calc_ent(sub_y)
        ent += (float(sub_y.shape[0]) / y.shape[0]) * temp_ent
    return ent

def calc_ent_grap(x,y): #计算信息增益
    base_ent = calc_ent(y)
    condition_ent = calc_condition_ent(x, y)
    ent_grap = base_ent - condition_ent

    return ent_grap

def select(Pop): #tournament选择
    selectIndex = random.sample(range(0,num), k)
    return Pop.iloc[selectIndex,:].sort_values(by="score",ascending=False).iloc[0,:]

def fitness(individual,trainData,label): #适应性函数
    #取出个体对应特征
    dropNames = []
    featureNames = trainData.columns
    for i in range(individual.shape[0]):
        if(individual[i] == 0):
            dropNames.append(featureNames[i])
    trainData = trainData.drop(dropNames,axis=1)
    #执行5折交叉验证
    sensitivity2 = []
    specificity2 = []
    kf = KFold(n_splits=5,shuffle=False)
    for train_index ,test_index in kf.split(trainData):
        X_train = trainData.iloc[train_index,:]
        X_test = trainData.iloc[test_index,:]
        Y_train = np.array(label)[train_index]
        Y_test = np.array(label)[test_index]
        gnb = GaussianNB()
        y_pred = gnb.fit(X_train, Y_train).predict(X_test)
        sensi,speci = GM_score(y_pred,Y_test)
        sensitivity2.append(sensi)
        specificity2.append(speci)
    a = np.nanmean(sensitivity2)
    b = np.nanmean(specificity2)
        
    return math.sqrt(a*b)
    
def GM_score(x,y): #计算GM 传入numpy
    cm1 = confusion_matrix(x,y)
    sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    return sensitivity1,specificity1

def GeneticAlgorithm(trainData,label): #遗传算法
    Pop = pd.DataFrame(np.random.randint(0,2,(num,d)))#Pop:当前种群
    Pop["score"] = 0
    for i in range(G):
        newPop = Pop.copy()#初始化newPop
        #评估每个个体
        Pop["score"] = Pop.iloc[:,:-1].apply(fitness,args=(trainData,label),axis=1)

        #选出j(使剩余个体为偶数)个精英
        Pop = Pop.sort_values(by="score",ascending=False)
        if(((num-elite) % 2) != 0):
            j = elite + 1
        else:
            j = elite
        for k in range(j):
            newPop.iloc[k,:] = Pop.iloc[k,:]

        #正常遗传，交叉、变异
        for k in range(int((num-elite)/2)): 
            a = select(Pop).copy()#根据个体适应度，使用tournament算法选择Pop中的2个个体
            b = select(Pop).copy()
            
            for index in range(d):
                if(random.random() < Pc):#交叉
                    temp = a[index]
                    a[index] = b[index]
                    b[index] = temp
            Pm = Pm_standard  #执行普通遗传算法时，突变概率为定值
            for index in range(d):
                #Pm = SHEMutation(a,index)#根据算法求解变异概率
                Pm = CbHEMutation(trainData,b,b.index[index])
                if(random.random() < Pm):#变异a
                    a[index] = 0 if a[index] == 1 else 1
    
                #Pm = SHEMutation(b,index)#根据算法求解变异概率
                Pm = CbHEMutation(trainData,b,b.index[index])
                if(random.random() < Pm):#变异b
                    b[index] = 0 if b[index] == 1 else 1
            newPop.iloc[j,:] = a#加入newPop
            j = j + 1
            newPop.iloc[j,:] = b
            j = j + 1
        #用newPop替代Pop
        try:
            assert_frame_equal(Pop,newPop)
        except BaseException:
            pass
        else:
            break
        Pop = newPop.copy()
        print(Pop)
    #进化完之后根据score获得最佳个体
    Pop["score"] = Pop.iloc[:,:-1].apply(fitness,args=(trainData,label),axis=1)
    #选出j(使剩余个体为偶数)个精英
    Pop = Pop.sort_values(by="score",ascending=False)
    
    return Pop.iloc[0,:-1]
    
def getAncestors(index):
    global relativeList
    for relative in range(DAG.shape[0]):
        if(DAG.iloc[relative,index] == 1):
            relativeList.append(relative)
            getAncestors(relative)

def getDescendants(index):
    global relativeList
    for relative in range(DAG.shape[0]):
        if(DAG.iloc[index,relative] == 1):
            relativeList.append(relative)
            getAncestors(relative)
            
def SU(x,y): #计算对称不确定性系数
    return ((Entropy(x)+Entropy(y)-calc_condition_ent(x.values,y.values))/(Entropy(x)+Entropy(y)))

def SHEMutation(indiv,index):
    global relativeList
    if(indiv[index] == 0):
        return Pm_standard
    #先找到当前节点的所有祖先或后代节点
    relativeList = []
    getDescendants(index)
    getAncestors(index)
    relativeSet = set(relativeList)
    for relative_index in relativeSet:
        if(indiv[relative_index] == 1):
            return SHEP
    return Pm_standard

def CbHEMutation(trainData,indiv,index):
    corr = []
    if(indiv[index] == 0):
        return Pm_standard
    #先找到当前节点的所有祖先或后代节点
    relativeList = []
    getDescendants(index)
    getAncestors(index)
    relativeSet = set(relativeList)
    for relative_index in relativeSet:
        if(indiv[relative_index] == 1):
            corr.append(SUMetrics.iloc[index,relative_index])
    return np.mean(corr)

In [75]:
sensitivity = []
specificity = []
F1 = []
AUC = []
#10折交叉验证
kf = KFold(n_splits=5,shuffle=False)
for train_index ,test_index in kf.split(data):
    for i in range(data.shape[1]):
        for j in range(data.shape[1]):
            SUMetrics.iloc[i,j] = SU(data.iloc[train_index,:].iloc[:,i],data.iloc[train_index,:].iloc[:,j])
    print(SUMetrics)
    #for i in range(10): #设置10次随机种子，每折执行10次
    #random.seed(i)
    bestIndiv = GeneticAlgorithm(data.iloc[train_index,:].copy(),np.array(label)[train_index].copy()) #执行遗传算法，获得最优特征组合个体
    #通过最优个体，选出最优特征组合
    dropNames = []
    featureNames = data.columns
    for i in range(bestIndiv.shape[0]):
        if(bestIndiv[i] == 0):
            dropNames.append(featureNames[i])
    tempData = data.drop(dropNames,axis=1).copy() #tempData用于保存此次用于测试的特征集
    #训练模型,计算结果
    X_train = tempData.iloc[train_index,:]
    X_test = tempData.iloc[test_index,:]
    Y_train = np.array(label)[train_index]
    Y_test = np.array(label)[test_index]
    print(X_train.shape)
    print(X_test.shape)
    print(Y_train.shape)
    print(Y_test.shape)
    gnb = GaussianNB()
    y_pred = gnb.fit(X_train, Y_train).predict(X_test)
    
    sensi,speci= GM_score(y_pred,Y_test)
    sensitivity.append(sensi)
    specificity.append(speci)
    F1.append(f1_score(y_pred,Y_test))
    AUC.append(roc_auc_score(y_pred,Y_test))
    
a = np.nanmean(sensitivity)
b = np.nanmean(specificity)
print(a)
print(b)
print("GM")
print(math.sqrt(a*b))
print("f1")
print(np.nanmean(F1))
print("AUC")
print(np.nanmean(AUC))

<ipython-input-74-53402ad317b1>:173: RuntimeWarning: invalid value encountered in double_scalars
  return ((Entropy(x)+Entropy(y)-calc_condition_ent(x.values,y.values))/(Entropy(x)+Entropy(y)))


            GO:0006006  GO:0006629  GO:0006749  GO:0032094  GO:0034599  \
GO:0006006    1.000000    0.580886    0.634859    0.644698    0.519878   
GO:0006629    0.580886    1.000000    0.634859    0.569120    0.519878   
GO:0006749    0.442893    0.442893    1.000000    0.495417    0.442893   
GO:0032094    0.560981    0.485402    0.605434    1.000000    0.485402   
GO:0034599    0.519878    0.519878    0.634859    0.569120    1.000000   
...                ...         ...         ...         ...         ...   
GO:0034605    0.408476    0.442893    0.503278    0.448901    0.442893   
GO:0031929    0.408476    0.408476    0.503278    0.448901    0.408476   
GO:0043087    0.333470    0.333470    0.423493    0.371209    0.333470   
GO:2000774    0.408476    0.408476    0.503278    0.448901    0.408476   
GO:0006325    0.408476    0.408476    0.503278    0.448901    0.408476   

            GO:0043523  GO:0045471  GO:0055114  GO:2000378  GO:0006468  ...  \
GO:0006006    0.569120    0.6348

c:\users\li\appdata\local\programs\python\python38\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\users\li\appdata\local\programs\python\python38\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


       0    1    2    3    4    5    6    7    8    9  ...  317  318  319  \
0    1.0  0.0  1.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  ...  1.0  1.0  1.0   
1    1.0  0.0  1.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  ...  1.0  1.0  1.0   
2    1.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  ...  0.0  0.0  1.0   
3    1.0  1.0  1.0  1.0  1.0  1.0  0.0  1.0  0.0  1.0  ...  1.0  0.0  0.0   
4    1.0  1.0  1.0  0.0  1.0  0.0  0.0  1.0  1.0  1.0  ...  1.0  0.0  1.0   
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
141  1.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  ...  0.0  0.0  1.0   
142  1.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  ...  0.0  0.0  0.0   
143  0.0  1.0  1.0  1.0  0.0  0.0  1.0  0.0  1.0  1.0  ...  1.0  0.0  1.0   
144  0.0  0.0  0.0  1.0  1.0  0.0  1.0  1.0  0.0  0.0  ...  0.0  1.0  0.0   
145  0.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  ...  1.0  1.0  1.0   

     320  321  322  323  324  325     score  
0    1.0  0.0  0.0  0.0  1.0 

c:\users\li\appdata\local\programs\python\python38\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\users\li\appdata\local\programs\python\python38\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


...

       0    1    2    3    4    5    6    7    8    9  ...  317  318  319  \
0    1.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0  0.0  1.0  ...  0.0  1.0  0.0   
1    1.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...  0.0  1.0  0.0   
2    1.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...  1.0  1.0  0.0   
3    1.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...  0.0  1.0  0.0   
4    1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...  0.0  1.0  0.0   
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
141  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...  0.0  1.0  1.0   
142  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...  1.0  1.0  0.0   
143  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...  1.0  1.0  0.0   
144  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...  1.0  1.0  0.0   
145  1.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0  0.0  1.0  ...  0.0  1.0  0.0   

     320  321  322  323  324  325     score  
0    1.0  1.0  1.0  0.0  1.0 

c:\users\li\appdata\local\programs\python\python38\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\users\li\appdata\local\programs\python\python38\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


       0    1    2    3    4    5    6    7    8    9  ...  317  318  319  \
0    1.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0  0.0  1.0  ...  0.0  1.0  0.0   
1    1.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...  0.0  1.0  0.0   
2    1.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...  0.0  1.0  0.0   
3    1.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...  0.0  1.0  0.0   
4    1.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...  0.0  1.0  0.0   
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
141  1.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...  0.0  1.0  0.0   
142  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  1.0  ...  0.0  1.0  0.0   
143  1.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...  0.0  1.0  0.0   
144  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...  1.0  1.0  0.0   
145  1.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...  0.0  1.0  0.0   

     320  321  322  323  324  325     score  
0    1.0  1.0  1.0  0.0  1.0 

c:\users\li\appdata\local\programs\python\python38\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\users\li\appdata\local\programs\python\python38\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


KeyboardInterrupt: 

In [76]:
f1_score(y_pred,Y_test)

0.3939393939393939
0.7416666666666667
GM
0.5405291085332197
f1
0.7283653846153846
AUC


In [9]:
Pop = pd.DataFrame(np.random.randint(0,2,(5,10)))

Pop["sum"] = Pop.apply(get,args=(9,9),axis=1)

3
9 and 9
6
9 and 9
7
9 and 9
4
9 and 9
5
9 and 9


In [8]:
data["GO:0097190"].sum()


3